In [ ]:
#!g1.1
!git clone https://github.com/openai/CLIP
%pip install taming-transformers
!git clone https://github.com/CompVis/taming-transformers.git
%pip install ftfy regex tqdm omegaconf pytorch-lightning
%pip install kornia 
%pip install imageio-ffmpeg   
%pip install einops

In [17]:
#!g1.1
%pip install gradio

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [18]:
#!g1.1
%pip install imageio

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 3.4 MB 3.2 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [1]:
#!g1.1
%load_ext autoreload
%autoreload 2

In [1]:
#!g1.1

import numpy as np
from PIL import Image
from torchvision.transforms import ToPILImage
# from generate_from_clip import generate

import os
os.sys.path.append('taming_transformers')
import yaml
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader

from taming.models.vqgan import VQModel
from CLIP import clip

import math
import torch
from torch import nn
import torch.nn.functional as F
import kornia.augmentation as K

from torchvision import transforms
from torchvision.transforms import functional as TF

import matplotlib.pyplot as plt
from PIL import Image

from tqdm.auto import tqdm
import imageio

from IPython.display import clear_output

In [2]:
#!g1.1
import gradio as gr
# !mkdir vqgan
# !curl -L -o vqgan/config.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1'
# !curl -L -o vqgan/model.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' 

In [3]:
#!g1.1
from cutouts import MakeCutouts

In [4]:
#!g1.1
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)

replace_grad = ReplaceGrad.apply

class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)

    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None


clamp_with_grad = ClampWithGrad.apply


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

normalize = transforms.Normalize(
    mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]
)


# model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor, preprocess = clip.load('ViT-B/32', device=device)
perceptor = perceptor.eval().requires_grad_(False)

cut_size = perceptor.visual.input_resolution
make_cutouts = MakeCutouts(cut_size, cutn=32)

def load_vqgan_model(config_path, checkpoint_path):
    config = yaml.load(open(config_path, 'r'), Loader)

    model = VQModel(**config['model']['params'])
    model.eval().requires_grad_(False)
    model.init_from_ckpt(checkpoint_path)

    del model.loss
    return model

vqgan = load_vqgan_model('vqgan/config.yaml', 'vqgan/model.ckpt').to(device)

def vector_quantize(x, codebook):
    # dist [bs, size, size, n_tokens]: (x - codebook)^2 = x^2 + codebook^2 - 2 * x * codebook
    dist = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = dist.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(dist.dtype) @ codebook
    return replace_grad(x_q, x)

# constants
N_TOKENS = vqgan.quantize.n_e
EMB_DIM = vqgan.quantize.e_dim


def generate(prompt_text=None, prompt_image=None, negative_prompt=None, starting_img=None, n_images=5, n_iters=100, lr=0.2):
    prompts = []
    if len(prompt_text) != 0:
        prompt_encoding = perceptor.encode_text(clip.tokenize(prompt_text).to(device)).float()
        prompt_encoding_normed = F.normalize(prompt_encoding.unsqueeze(0) + 1e-6, dim=2)
        prompts.append(prompt_encoding_normed)
    if prompt_image is not None:
        w, h = prompt_image.size
        pad = transforms.Pad(
            [(max(w, h) - w) // 2, (max(w, h) - h) // 2],
            fill=0, padding_mode='constant'
        )
        transform = transforms.Compose([pad, transforms.ToTensor(), transforms.Resize(224), normalize])
        image = transform(prompt_image).unsqueeze(0).to(device)
        prompt_encoding = perceptor.encode_image(image).float()
        image_encoding_normed = F.normalize(prompt_encoding.unsqueeze(0) + 1e-6, dim=2)
        prompts.append(image_encoding_normed)
        
    if len(negative_prompt) != 0:
        prompt_encoding = perceptor.encode_text(clip.tokenize(negative_prompt).to(device)).float()
        negative_prompt_normed = F.normalize(prompt_encoding.unsqueeze(0) + 1e-6, dim=2)

    if starting_img is not None:
        w, h = starting_img.size

        pad = transforms.Pad(
            [(max(w, h) - w) // 2, (max(w, h) - h) // 2],
            fill=0, padding_mode='constant'
        )
    
        transform = transforms.Compose([pad, transforms.ToTensor(), transforms.Resize(256), normalize])
        input_image = transform(starting_img).to(device)
        z = vqgan.encode(input_image.unsqueeze(0))[0]

        z = z.repeat(n_images, 1, 1, 1).requires_grad_(True)
    else:
        one_hot = F.one_hot(torch.randint(N_TOKENS, [n_images, 16 * 16], device=device), N_TOKENS).float()
        z = one_hot @ vqgan.quantize.embedding.weight
        z = z.view([-1, 16, 16, EMB_DIM]).permute(0, 3, 1, 2).requires_grad_(True)

    optimizer = torch.optim.Adam([z], lr=lr)
    
    z_q = vector_quantize(z.movedim(1, 3), vqgan.quantize.embedding.weight).movedim(3, 1)
    out = clamp_with_grad((vqgan.decode(z_q) + 1) / 2, 0, 1)
    for i in tqdm(range(n_iters)):
        z_q = vector_quantize(z.movedim(1, 3), vqgan.quantize.embedding.weight).movedim(3, 1)
        out = clamp_with_grad((vqgan.decode(z_q) + 1) / 2, 0, 1)
        back_encoding = perceptor.encode_image(normalize(make_cutouts(out))).float()

        input_normed = F.normalize(back_encoding.unsqueeze(1) + 1e-6, dim=2)
        dists = 0
        for prompt_encoding in prompts:
            dists += input_normed.sub(prompt_encoding).norm(dim=2).div(2).arcsin().pow(2).mul(2)

        if len(negative_prompt) != 0:
            dists -= input_normed.sub(negative_prompt_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)

        loss = dists.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
        img = np.transpose(img, (1, 2, 0))
        imageio.imwrite('./steps/' + str(i) + '.png', np.array(img))

    images = [ToPILImage()(image.clamp(0, 1)) for image in out.detach().cpu()]
    return images

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /tmp/xdg_cache/torch/hub/checkpoints/vgg16-397923af.pth



loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Restored from vqgan/model.ckpt


In [ ]:
#!g1.1
def run(prompt_text, prompt_image, negative_prompt, starting_img, n_images, n_iters, lr):
    if lr is None:
        lr = 0.2
    lr = max(0, lr)
    if len(prompt_text) != 0 or prompt_image is not None:
        images = generate(
            prompt_text=prompt_text,
            prompt_image=prompt_image,
            negative_prompt=negative_prompt,
            starting_img=starting_img,
            n_images=n_images,
            n_iters=n_iters,
            lr=lr
        )
        video_ = generate_video(n_iters)
        return images, video_
    

def generate_video(n_iters):
    init_frame = 1
    last_frame = n_iters

    min_fps = 10
    max_fps = 60

    total_frames = last_frame - init_frame

    length = 15

    frames = []
    tqdm.write('Generating video...')
    for i in range(init_frame, last_frame): #
        frames.append(Image.open("./steps/" + str(i) + '.png'))

    #fps = last_frame/10
    fps = np.clip(total_frames / length, min_fps, max_fps)

    from subprocess import Popen, PIPE
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
    for im in tqdm(frames):
        im.save(p.stdin, 'PNG')
    p.stdin.close()
    p.wait()
    return 'video.mp4'


height = 256
width = 256

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt_image = gr.Image(label='Prompt image', type="pil")
            prompt_text = gr.Textbox(label='Prompt text', placeholder="Request Here...")
            negative_prompt = gr.Textbox(label='Negative prompt', placeholder="Request Here...")
        with gr.Column():
            startimg_image = gr.Image(label='Starting image', type="pil")
            n_images = gr.Slider(1, 5, step=1, value=3, label='Number of output images')
            n_iters = gr.Slider(0, 500, step=10, value=100, label='Number of iterations')
    with gr.Accordion("Advanced options", open=False):
        lr = gr.Number(label='Learning rate', value=0.2)
    with gr.Row():
        run_btn = gr.Button(value="Generate")
    with gr.Row():
        gallery=gr.Gallery(show_label=False)
        gallery.style(grid=5)
        video = gr.Video()

    run_btn.click(
        run,
        inputs=[prompt_text, prompt_image, negative_prompt, startimg_image, n_images, n_iters, lr],
        outputs=[gallery, video],
        api_name="generate-from-clip"
    )

demo.launch(debug=True, share=True, enable_queue=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3d2171fd-7e45-440c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
#!g1.1


In [ ]:
#!g1.1


In [ ]:
#!g1.1
# img = Image.open('IMG_20221221_152235_535.jpg')
out = generate(prompt_text='scary house, halloween, evening lightning, neo lighting,\
          ultra-detailed, ultrarealistic, photorealism, 8k', prompt_image=None, starting_img=None, n_images=3, n_iters=100, lr=0.1)

In [ ]:
#!g1.1
out[0]

In [ ]:
#!g1.1
plt.figure(figsize=(18, 8))
for i in range(3):
    plt.subplot(1, 5, i + 1)
#     img = out[i].permute(1, 2, 0)
    plt.axis('off')
    plt.tight_layout()

    plt.imshow(out[i])

In [ ]:
#!g1.1
